In [55]:
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib

import pandas as pd
import numpy as np

/opt/conda/python37/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [62]:
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 50

In [56]:
df = pd.read_csv('White_ЦИАН_Мск_МО.csv')
df.columns

Index(['cian_id', 'photo', 'url', 'bargainterms_price',
       'bargainterms_price_LN', 'area_total', 'area_total_1', 'unit_price',
       'unit_price_LN', 'rooms_count', 'rooms_count_1', 'floor_1', 'floor',
       'floors_count', 'wall_type', 'last_edited', 'repair_type', 'lat', 'lng',
       'lat_lng', 'geo_region', 'geo_settlement_1', 'geo_city_okrug_1',
       'geo_city_district_1', 'geo_street', 'geo_house_num',
       'distance_to_centre', 'distance_to_centre_1', 'rings_MSK', 'rings_MKAD',
       'subway_name', 'subway_travel_type', 'subway_time_1', 'subway_time',
       'nearest_metro', 'distance_to_nearest_metro', 'description', 'ptn_dadd',
       'Регион_enc', 'med_price', 'Районы_Kmeans'],
      dtype='object')

In [ ]:
df.head(10)

In [81]:
df_elit = pd.read_excel('БД_Элитка_Москва.xlsx', sheet_name = 0)
df_elit.columns

Index(['№', 'Наименование ЖК', 'Район города', 'Нименование улицы',
       'Номер дома', 'Адрес', 'Широта', 'Долгота', 'Сцепка_к', 'Тип', 'Статус',
       'Видовые характеристики', 'Конкретика', 'Витрины',
       'Диапазон стоимости, руб./кв. м', 'Источник информации цен',
       'Ссылка на сайт ЖК'],
      dtype='object')

In [ ]:
df_elit

In [58]:
s_1 = df['unit_price'] >= 500000

df_sort = df[s_1] 
df_sort.reset_index(drop = True , inplace = True )
len(df_sort)

9153

In [66]:
address = []

for i in tqdm(range(len(df_sort))):
    if df_sort['geo_region'][i] == 'Московская область':
        address.append(f"{df_sort['geo_region'][i]}, {df_sort['geo_city_district_1'][i]}, {df_sort['geo_street'][i]}, {df_sort['geo_house_num'][i]}")
    else:
        address.append(f"{df_sort['geo_region'][i]}, {df_sort['geo_street'][i]}, {df_sort['geo_house_num'][i]}")
        
df_sort['address_full'] = address
df_sort.columns

100%|██████████| 9153/9153 [00:00<00:00, 27234.50it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Index(['cian_id', 'photo', 'url', 'bargainterms_price',
       'bargainterms_price_LN', 'area_total', 'area_total_1', 'unit_price',
       'unit_price_LN', 'rooms_count', 'rooms_count_1', 'floor_1', 'floor',
       'floors_count', 'wall_type', 'last_edited', 'repair_type', 'lat', 'lng',
       'lat_lng', 'geo_region', 'geo_settlement_1', 'geo_city_okrug_1',
       'geo_city_district_1', 'geo_street', 'geo_house_num',
       'distance_to_centre', 'distance_to_centre_1', 'rings_MSK', 'rings_MKAD',
       'subway_name', 'subway_travel_type', 'subway_time_1', 'subway_time',
       'nearest_metro', 'distance_to_nearest_metro', 'description', 'ptn_dadd',
       'Регион_enc', 'med_price', 'Районы_Kmeans', 'address_full'],
      dtype='object')

In [67]:
df_sort['address_full']

0            Москва, улица Крылатские Холмы, 37
1             Москва, Ленинградское шоссе, 37к1
2           Москва, улица Большая Полянка, 61С2
3       Москва, Садовая-Кудринская улица, 28-30
4                 Москва, улица Новый Арбат, 29
                         ...                   
9148            Москва, Ильменский проезд, 14к1
9149      Москва, Малый Каковинский переулок, 8
9150               Москва, Винницкая улица, 8к1
9151           Москва, Хорошевское шоссе, 25Ак2
9152                   Москва, улица Зорге, 9к2
Name: address_full, Length: 9153, dtype: object

In [108]:
table1 = df_sort[['cian_id', 'lat', 'lng', 'address_full']]
table1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9153 entries, 0 to 9152
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cian_id       9153 non-null   int64  
 1   lat           9153 non-null   float64
 2   lng           9153 non-null   float64
 3   address_full  9153 non-null   object 
dtypes: float64(2), int64(1), object(1)
memory usage: 286.2+ KB


In [109]:
table2 = df_elit[['Широта', 'Долгота', 'Адрес', 'Тип']]
table2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409 entries, 0 to 408
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Широта   409 non-null    float64
 1   Долгота  409 non-null    float64
 2   Адрес    409 non-null    object 
 3   Тип      409 non-null    object 
dtypes: float64(2), object(2)
memory usage: 12.9+ KB


In [110]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Преобразуем градусы в радианы
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Создаем массив для координат
    coords_1 = np.array([[lat1_rad, lon1_rad]])
    coords_2 = np.array([[lat2_rad, lon2_rad]])
    
    # Вычисляем расстояние
    return haversine_distances(coords_1, coords_2) * 6371000  # Умножаем на радиус Земли в метрах

In [111]:
# Создаем новый столбец для хранения индексов схожих объектов
table1['similar_indices'] = [[] for _ in range(len(table1))]
table1['distance'] = [[] for _ in range(len(table1))]
# Для сравнения адресов
table1['address_ratio'] = [[] for _ in range(len(table1))]
table1['address_partial_ratio'] = [[] for _ in range(len(table1))]
table1['address_token_sort_ratio'] = [[] for _ in range(len(table1))]
table1['address_token_set_ratio'] = [[] for _ in range(len(table1))]
table1['address_SeqMatcher_ratio'] = [[] for _ in range(len(table1))]
table1['address_cosine_sim'] = [[] for _ in range(len(table1))]

# Определяем порог расстояния (в метрах)
distance_threshold = 80 #26

# Вычисляем расстояния и заполняем столбец
for i, row1 in tqdm(table1.iterrows(), total = table1.shape[0], position = 0):
    for j, row2 in table2.iterrows():
        distance = calculate_distance(row1['lat'], row1['lng'], row2['Широта'], row2['Долгота'])
        if distance < distance_threshold:
            
            ratio = fuzz.ratio(table1['address_full'][i], table2['Адрес'][j])                         # простой алгоритм схожести
            partial_ratio = fuzz.partial_ratio(table1['address_full'][i], table2['Адрес'][j])         # хорошо ищет, когда в адресе есть пропуски (наилучш. частич. совпадение)
            token_sort_ratio = fuzz.token_sort_ratio(table1['address_full'][i], table2['Адрес'][j])   # разбивает на токены, сортирует, хорош когда есть перестановка слов
            token_set_ratio = fuzz.token_set_ratio(table1['address_full'][i], table2['Адрес'][j])     # аналогичен предыдущему, но игнорирует дубли токенов
            
            SequenceMatcher_ratio = difflib.SequenceMatcher(None, table1['address_full'][i].lower(), table2['Адрес'][j].lower()).ratio() # схожесть последовательностей
            
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform([table1['address_full'][i], table2['Адрес'][j]])   # косинусное сходство между векторами
            cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
                     
            table1.at[i, 'similar_indices'].append(j)
            table1.at[i, 'distance'].append(round(distance[0][0], 1))
            table1.at[i, 'address_ratio'].append(ratio)
            table1.at[i, 'address_partial_ratio'].append(partial_ratio)
            table1.at[i, 'address_token_sort_ratio'].append(token_sort_ratio)
            table1.at[i, 'address_token_set_ratio'].append(token_set_ratio)
            table1.at[i, 'address_SeqMatcher_ratio'].append(round(SequenceMatcher_ratio, 2))
            table1.at[i, 'address_cosine_sim'].append(round(cosine_sim, 2))

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [112]:
table1

,cian_id,lat,lng,address_full,similar_indices,distance,address_ratio,address_partial_ratio,address_token_sort_ratio,address_token_set_ratio,address_SeqMatcher_ratio,address_cosine_sim
0,239293843,55.76035,37.41940,"Москва, улица Крылатские Холмы, 37",[],[],[],[],[],[],[],[]
1,239383173,55.83210,37.48396,"Москва, Ленинградское шоссе, 37к1",[],[],[],[],[],[],[],[]
2,239000665,55.73099,37.62307,"Москва, улица Большая Полянка, 61С2",[],[],[],[],[],[],[],[]
3,239124088,55.76459,37.58984,"Москва, Садовая-Кудринская улица, 28-30",[],[],[],[],[],[],[],[]
4,239001047,55.75146,37.57736,"Москва, улица Новый Арбат, 29",[382],[0.4],[72],[72],[100],[100],[0.72],[1.0]
...,...,...,...,...,...,...,...,...,...,...,...,...
9148,308037844,55.86003,37.54672,"Москва, Ильменский проезд, 14к1",[256],[51.9],[74],[74],[100],[100],[0.74],[1.0]
9149,308136577,55.75079,37.58508,"Москва, Малый Каковинский переулок, 8",[],[],[],[],[],[],[],[]
9150,308163399,55.70098,37.49347,"Москва, Винницкая улица, 8к1",[],[],[],[],[],[],[],[]
9151,308343072,55.77531,37.52724,"Москва, Хорошевское шоссе, 25Ак2",[166],[0.4],[72],[72],[97],[97],[0.72],[0.6]


In [113]:
filtered_data = table1[table1['similar_indices'].apply(lambda x: len(x) > 0)]
filtered_data = filtered_data[filtered_data['address_token_set_ratio'].apply(lambda x: x[0] > 83)]
filtered_data.reset_index(drop = True , inplace = True )
filtered_data

,cian_id,lat,lng,address_full,similar_indices,distance,address_ratio,address_partial_ratio,address_token_sort_ratio,address_token_set_ratio,address_SeqMatcher_ratio,address_cosine_sim
0,239001047,55.75146,37.57736,"Москва, улица Новый Арбат, 29",[382],[0.4],[72],[72],[100],[100],[0.72],[1.0]
1,235565015,55.71600,37.50677,"Москва, Минская улица, 2Вк1",[25],[0.3],[70],[70],[100],[100],[0.7],[1.0]
2,233755593,55.72067,37.68934,"Москва, Волгоградский проспект, 32/5к1",[281],[64.1],[76],[76],[97],[97],[0.76],[0.67]
3,235048570,55.74076,37.58105,"Москва, 1-й Неопалимовский переулок, 8",[145],[0.6],[79],[79],[100],[100],[0.79],[1.0]
4,235191953,55.76925,37.60230,"Москва, Старопименовский переулок, 10",[401],[12.8],[78],[78],[100],[100],[0.78],[1.0]
...,...,...,...,...,...,...,...,...,...,...,...,...
2729,308100502,55.69202,37.48194,"Москва, улица Сергея Бондарчука, 4","[16, 17]","[45.6, 48.2]","[76, 74]","[76, 74]","[100, 97]","[100, 97]","[0.76, 0.74]","[1.0, 1.0]"
2730,308202546,55.76459,37.51128,"Москва, Шелепихинская набережная, 34к5",[86],[0.4],[79],[79],[100],[100],[0.79],[1.0]
2731,308614305,55.76395,37.50979,"Москва, Шелепихинская набережная, 34к6",[85],[0.5],[79],[79],[100],[100],[0.79],[1.0]
2732,308037844,55.86003,37.54672,"Москва, Ильменский проезд, 14к1",[256],[51.9],[74],[74],[100],[100],[0.74],[1.0]


In [116]:
addr = []
tip = []

for i in range(len(filtered_data)):
    ind = [filtered_data['similar_indices'][i][0]]
    addr.append(table2.iloc[ind]['Адрес'].values[0])
    tip.append(table2.iloc[ind]['Тип'].values[0])

In [119]:
table_final = filtered_data[['cian_id', 'lat', 'lng', 'address_full', 'similar_indices', 'address_token_set_ratio']]
table_final['address_elit'] = addr
table_final['tip'] = tip
table_final

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,cian_id,lat,lng,address_full,similar_indices,address_token_set_ratio,address_elit,tip
0,239001047,55.75146,37.57736,"Москва, улица Новый Арбат, 29",[382],[100],"улица Новый Арбат, 29, Москва",престижный
1,235565015,55.71600,37.50677,"Москва, Минская улица, 2Вк1",[25],[100],"Минская улица, 2Вк1, Москва",престижный+
2,233755593,55.72067,37.68934,"Москва, Волгоградский проспект, 32/5к1",[281],[97],"Волгоградский проспект, 32/5к3, Москва",престижный
3,235048570,55.74076,37.58105,"Москва, 1-й Неопалимовский переулок, 8",[145],[100],"1-й Неопалимовский переулок, 8, Москва",престижный
4,235191953,55.76925,37.60230,"Москва, Старопименовский переулок, 10",[401],[100],"Старопименовский переулок, 10, Москва",престижный+
...,...,...,...,...,...,...,...,...
2729,308100502,55.69202,37.48194,"Москва, улица Сергея Бондарчука, 4","[16, 17]","[100, 97]","улица Сергея Бондарчука, 4, Москва",престижный
2730,308202546,55.76459,37.51128,"Москва, Шелепихинская набережная, 34к5",[86],[100],"Шелепихинская набережная, 34к5, Москва",престижный
2731,308614305,55.76395,37.50979,"Москва, Шелепихинская набережная, 34к6",[85],[100],"Шелепихинская набережная, 34к6, Москва",престижный
2732,308037844,55.86003,37.54672,"Москва, Ильменский проезд, 14к1",[256],[100],"Ильменский проезд, 14к1, Москва",престижный


In [124]:
mapping = dict(zip(table_final['cian_id'], table_final['tip']))
df['type'] = df['cian_id'].apply(lambda x: mapping.get(x))

mapping = dict(zip(table_final['cian_id'], table_final['address_elit']))
df['address_elit'] = df['cian_id'].apply(lambda x: mapping.get(x))

df['type'] = np.where(df['type'].isnull(), 'нет', df['type'])
df['address_elit'] = np.where(df['address_elit'].isnull(), 'нет', df['address_elit'])

df['type'].value_counts()

нет            65831
престижный      1484
престижный+     1250
Name: type, dtype: int64

In [125]:
df['address_elit'].value_counts()

нет                                       65831
Шелепихинская набережная, 40к1, Москва       93
улица Усачёва, 15А, Москва                   89
Казарменный переулок, 3, Москва              79
Шелепихинская набережная, 34к5, Москва       69
                                          ...  
улица Арбат, 39, Москва                       1
Гранатный переулок, 8с4, Москва               1
Ордынский тупик, 4, Москва                    1
улица Большая Дмитровка, 7/5с2, Москва        1
Островной проезд, 12к1, Москва                1
Name: address_elit, Length: 188, dtype: int64

In [98]:
# Выбор первого объекта и его похожих объектов
first_object_index = 0
similar_indices = filtered_data['similar_indices'][first_object_index]

# Создание новой таблицы с похожими объектами
similar_objects_df = table2.iloc[similar_indices].copy()

# Вывод информации
print("Первый объект:")
print(filtered_data.iloc[first_object_index])
print("\nПохожие объекты:")
print(similar_objects_df)

Первый объект:
lat                                              55.75146
lng                                              37.57736
address_full                Москва, улица Новый Арбат, 29
similar_indices                                     [382]
distance                                            [0.4]
address_ratio                                        [72]
address_partial_ratio                                [72]
address_token_sort_ratio                            [100]
address_token_set_ratio                             [100]
address_SeqMatcher_ratio                           [0.72]
address_cosine_sim                                  [1.0]
Name: 0, dtype: object

Похожие объекты:
        Широта    Долгота                          Адрес         Тип
382  55.751462  37.577355  улица Новый Арбат, 29, Москва  престижный


In [126]:
df.columns

Index(['cian_id', 'photo', 'url', 'bargainterms_price',
       'bargainterms_price_LN', 'area_total', 'area_total_1', 'unit_price',
       'unit_price_LN', 'rooms_count', 'rooms_count_1', 'floor_1', 'floor',
       'floors_count', 'wall_type', 'last_edited', 'repair_type', 'lat', 'lng',
       'lat_lng', 'geo_region', 'geo_settlement_1', 'geo_city_okrug_1',
       'geo_city_district_1', 'geo_street', 'geo_house_num',
       'distance_to_centre', 'distance_to_centre_1', 'rings_MSK', 'rings_MKAD',
       'subway_name', 'subway_travel_type', 'subway_time_1', 'subway_time',
       'nearest_metro', 'distance_to_nearest_metro', 'description', 'ptn_dadd',
       'Регион_enc', 'med_price', 'Районы_Kmeans', 'type', 'address_elit'],
      dtype='object')

In [127]:
df.to_csv('White_ЦИАН_Мск_МО_.csv')

In [128]:
df_rty = pd.read_csv('White_ЦИАН_Мск_МО.csv')
df_rty.columns

Index(['Unnamed: 0', 'cian_id', 'photo', 'url', 'bargainterms_price',
       'bargainterms_price_LN', 'area_total', 'area_total_1', 'unit_price',
       'unit_price_LN', 'rooms_count', 'rooms_count_1', 'floor_1', 'floor',
       'floors_count', 'wall_type', 'last_edited', 'repair_type', 'lat', 'lng',
       'lat_lng', 'geo_region', 'geo_settlement_1', 'geo_city_okrug_1',
       'geo_city_district_1', 'geo_street', 'geo_house_num',
       'distance_to_centre', 'distance_to_centre_1', 'rings_MSK', 'rings_MKAD',
       'subway_name', 'subway_travel_type', 'subway_time_1', 'subway_time',
       'nearest_metro', 'distance_to_nearest_metro', 'description', 'ptn_dadd',
       'Регион_enc', 'med_price', 'Районы_Kmeans', 'type', 'address_elit'],
      dtype='object')

In [ ]:
df_rty